In [ ]:
import numpy as np
import pandas as pd
import iode as io

# Load and merge UNData into one dataframe
pathwork = "To define"
file = "Omnia_Energy_Balance_v20250212_trav.xlsx"
pathdata = pathwork+file

Omnia_raw = pd.read_excel(pathdata, sheet_name = "ALL")

In [2]:
Omnia_trav = Omnia_raw.groupby(["Sector", "Sub-sector", "Enerp NeW", "Unit", "Year", "Region New"], as_index=False)["Value"].sum().copy()

In [ ]:
df_omnia = pd.DataFrame()

# DFINENP
lst_subsec_dfinen = ["AGR","IND","RES","SRV", "TRA", "OTH", "Own uses"]
Omnia_dfinen = Omnia_trav.loc[Omnia_trav["Sector"].isin(lst_subsec_dfinen)].copy()
Omnia_dfinen["Variable_NeW"] = "DFINENP" + Omnia_dfinen["Enerp NeW"]
Omnia_dfinen.drop(columns=["Sector", "Enerp NeW"], inplace = True)
Omnia_dfinen.rename(columns={"Sub-sector":  "Sector", "Region New": "Region_NeW"}, inplace = True)
Omnia_dfinen = Omnia_dfinen[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]

# DINTEN
lst_subsec_dinten = ["TRF-IN"]
Omnia_dinten = Omnia_trav.loc[Omnia_trav["Sector"].isin(lst_subsec_dinten)].copy()
Omnia_dinten["Variable_NeW"] = "DINTENP" + Omnia_dinten["Enerp NeW"]
Omnia_dinten.drop(columns=["Sector", "Enerp NeW"], inplace = True)
Omnia_dinten.rename(columns={"Sub-sector":  "Sector", "Region New": "Region_NeW"}, inplace = True)
Omnia_dinten = Omnia_dinten[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]
Omnia_dinten.head()



In [ ]:
# DINTEN PG & HEAT
lst_subsec_dinten_pg = ["PWR"]
lst_cmd_todrop_dinten_pg = ["ELEC", "HEAT"]
lst_var_chp = ["CHP AUTO", "CHP MAIN"]

#########
# Calculate the Electricitry and Heat production from CHP (to split inputs thereafter)
chp_calc = Omnia_trav.loc[(Omnia_trav["Sub-sector"].isin(lst_var_chp)) 
                          & ((Omnia_trav["Enerp NeW"].isin(lst_cmd_todrop_dinten_pg)))]
chp_calc = chp_calc.groupby(by=["Enerp NeW", "Unit", "Year", "Region New"], as_index= False)["Value"].sum()
for enrp in lst_cmd_todrop_dinten_pg:
    chp_calc.loc[chp_calc["Enerp NeW"] == f"{enrp}", "Variable_NeW"] = f"PRODP{enrp}"

# Pivot dataframe to calculate share of electricity (and heat) in CHP
chp_calc_pivoted = chp_calc.pivot_table(index=["Unit", "Year", "Region New"],columns="Enerp NeW", values="Value").reset_index()
chp_calc_pivoted["Share_ELEC"] = chp_calc_pivoted["ELEC"]/(chp_calc_pivoted["ELEC"]+chp_calc_pivoted["HEAT"])

# Calculate DINTENP for CHP 
chp_selec = Omnia_trav.loc[(Omnia_trav["Sub-sector"].isin(lst_var_chp)) 
                          & (~(Omnia_trav["Enerp NeW"].isin(lst_cmd_todrop_dinten_pg)))]
chp_selec = chp_selec.groupby(by=["Enerp NeW", "Unit", "Year", "Region New"], as_index= False)["Value"].sum()
# chp_dintenp["Variable_New"] = "DINTENP" + chp_dintenp["Enerp NeW"]

# Merge dfs to calculate "DINTENP_PG" et "DINTENP_HT"
chp_merged = chp_selec.merge(chp_calc_pivoted, on=["Unit", "Year", "Region New"], how="left")
chp_merged["DINTENP_PG"] = chp_merged["Value"]*chp_merged["Share_ELEC"]
chp_merged["DINTENP_HT"] = chp_merged["Value"]*(1-chp_merged["Share_ELEC"])
chp_merged.drop(columns=["ELEC", "HEAT", "Value", "Share_ELEC"], inplace= True)

# Convert columuns "DINTENP_PG" & "DINTENP_HT" in rows
chp_dintenp = pd.melt(chp_merged, id_vars=["Enerp NeW", "Unit", "Year", "Region New"], value_vars=["DINTENP_PG", "DINTENP_HT"],
                var_name="Sector", value_name="Value")
chp_dintenp.loc[(chp_dintenp["Sector"] == "DINTENP_PG"), "Sector_agg"] = "PG"
chp_dintenp.loc[(chp_dintenp["Sector"] == "DINTENP_HT"), "Sector_agg"] = "HEAT"
chp_dintenp.drop(columns=["Sector"], inplace= True)
chp_dintenp = chp_dintenp[["Region New", "Sector_agg", "Enerp NeW", "Year", "Value", "Unit"]]


#########
# DINTENP (excep. CHP)
Omnia_dinten_selec = Omnia_trav.loc[(Omnia_trav["Sector"].isin(lst_subsec_dinten_pg)) 
                                 & (~(Omnia_trav["Enerp NeW"].isin(lst_cmd_todrop_dinten_pg))) 
                                 & (~(Omnia_trav["Sub-sector"].isin(lst_var_chp)))].copy()
dict_elec_ht = {'Electricity AUTO': "PG", 'Electricity MAIN': "PG", 'Heat AUTO': "HEAT", 'Heat MAIN': "HEAT"}
for var, cmd in dict_elec_ht.items():
    Omnia_dinten_selec.loc[Omnia_dinten_selec["Sub-sector"] == f"{var}", "Sector_agg"] = f"{cmd}"
Omnia_dinten_selec = Omnia_dinten_selec.groupby(by=["Region New", "Sector_agg", "Enerp NeW", "Year", "Unit"], as_index= False)["Value"].sum()


########
# Merge DINTENP CHP & Non-CHP¨
Omnia_dinten_tot = pd.concat([Omnia_dinten_selec, chp_dintenp])
Omnia_dinten_pg = Omnia_dinten_tot.groupby(by=["Region New", "Sector_agg", "Enerp NeW", "Year","Unit"], as_index = False)["Value"].sum()
Omnia_dinten_pg["Variable_NeW"] = "DINTENP" + Omnia_dinten_pg["Enerp NeW"]
Omnia_dinten_pg.drop(columns=["Enerp NeW"], inplace=True)
Omnia_dinten_pg.rename(columns={"Region New": "Region_NeW", "Sector_agg": "Sector"}, inplace= True)
Omnia_dinten_pg = Omnia_dinten_pg[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]

#########
# Merge DINTENP PG/HEAT with other DINTENP
Omnia_dinten = pd.concat([Omnia_dinten, Omnia_dinten_pg])

Omnia_dinten.head()

In [6]:
## PRODP ELEC & HEAT
Omnia_prodp_elecht_trav = Omnia_trav.loc[(Omnia_trav["Sector"] == "PWR") 
                          & (Omnia_trav["Enerp NeW"].isin(lst_cmd_todrop_dinten_pg))].copy()

Omnia_prodp_elecht_trav = Omnia_prodp_elecht_trav.groupby(by=["Region New", "Enerp NeW", "Year", "Unit"], as_index= False)["Value"].sum()
Omnia_prodp_elecht_trav["Variable_NeW"] = "PRODP"
Omnia_prodp_elecht_trav.loc[(Omnia_prodp_elecht_trav["Enerp NeW"] == "ELEC"), "Enerp NeW"] = "PG"
Omnia_prodp_elecht_trav.rename(columns={"Region New": "Region_NeW", "Enerp NeW": "Sector"}, inplace = True)
Omnia_prodp_elecht = Omnia_prodp_elecht_trav[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]
Omnia_prodp_elecht.head()

,Variable_NeW,Region_NeW,Sector,Year,Value,Unit
0,PRODP,AU+WA,PG,2019,837116.0,TJ
1,PRODP,AU+WA,HEAT,2019,0.0,TJ
2,PRODP,BR,PG,2019,596926.0,TJ
3,PRODP,BR,HEAT,2019,0.0,TJ
4,PRODP,CA,PG,2019,867839.0,TJ


In [ ]:
## PROP except ELEC & HEAT
Omnia_prodp_trav = Omnia_trav.loc[(Omnia_trav["Sector"] == "UPS-MIN") & (Omnia_trav["Sub-sector"] == "Primary production") ].copy()
Omnia_prodp_trav = Omnia_prodp_trav.groupby(by=["Region New", "Enerp NeW", "Year", "Unit"], as_index = False)["Value"].sum()
Omnia_prodp_trav.rename(columns={"Region New": "Region_NeW"}, inplace=True)
Omnia_prodp_trav["Variable_NeW"] = "PRODP" + Omnia_prodp_trav["Enerp NeW"]
Omnia_prodp_trav["Sector"] = "Production"

Omnia_prodp = Omnia_prodp_trav[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]


In [ ]:
Omnia_prodp["Variable_NeW"].unique()

In [9]:
## DNONENRP
Omnia_dnonenp_trav = Omnia_trav.loc[(Omnia_trav["Sector"] == "Non energy")].copy()
Omnia_dnonenp_trav =Omnia_dnonenp_trav.groupby(by=["Region New", "Sector", "Enerp NeW", "Unit", "Year"], as_index=False)["Value"].sum()
Omnia_dnonenp_trav.rename(columns={"Region New": "Region_NeW"}, inplace=True)
Omnia_dnonenp_trav["Variable_NeW"] = "DNONENP" + Omnia_dnonenp_trav["Enerp NeW"]
Omnia_dnonenp = Omnia_dnonenp_trav[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]


## IMPP/EXPP
Omnia_trade_trav = Omnia_trav.loc[(Omnia_trav["Sector"] == "TRADE")].copy()
Omnia_trade_trav = Omnia_trade_trav.groupby(by=["Region New", "Sub-sector", "Enerp NeW", "Unit", "Year"], as_index=False)["Value"].sum()
Omnia_trade_trav .loc[(Omnia_trade_trav["Sub-sector"] == "Export"), "Variable_NeW"] = "EXPP" + Omnia_trade_trav["Enerp NeW"]
Omnia_trade_trav .loc[(Omnia_trade_trav["Sub-sector"] == "Import"), "Variable_NeW"] = "IMPP" + Omnia_trade_trav["Enerp NeW"]
Omnia_trade_trav.drop(columns=["Sub-sector"])
Omnia_trade_trav.rename(columns={"Region New": "Region_NeW"}, inplace=True)
Omnia_trade_trav["Sector"] = Omnia_trade_trav["Enerp NeW"]
Omnia_trade = Omnia_trade_trav[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]


## DSTOCKP
lst_dstock = ["Consumption by stocks", "Production by stocks"]
Omnia_dstockp_trav = Omnia_trav.loc[(Omnia_trav["Sector"] == "UPS-MIN") & (Omnia_trav["Sub-sector"].isin(lst_dstock))].copy()
Omnia_dstockp_trav= Omnia_dstockp_trav.groupby(["Region New", "Enerp NeW", "Unit", "Year"], as_index=False)["Value"].sum()
Omnia_dstockp_trav["Variable_NeW"] = "DSTOCKP" + Omnia_dstockp_trav["Enerp NeW"]
Omnia_dstockp_trav["Sector"] = Omnia_dstockp_trav["Enerp NeW"]
Omnia_dstockp_trav.rename(columns={"Region New": "Region_NeW"}, inplace=True)
Omnia_dstockp = Omnia_dstockp_trav[["Variable_NeW", "Region_NeW", "Sector", "Year", "Value", "Unit"]]


In [ ]:
Omnia_eb = pd.concat([Omnia_dfinen, Omnia_dinten, Omnia_prodp_elecht, Omnia_prodp, Omnia_dnonenp, Omnia_trade, Omnia_dstockp])
Omnia_eb["Variable_NeW"].unique()
Omnia_eb.loc[(Omnia_eb["Variable_NeW"] == "PRODP") & (Omnia_eb["Sector"] == "PG"), "Variable_NeW"] = "PRODPELEC"
Omnia_eb.loc[(Omnia_eb["Variable_NeW"] == "PRODP") & (Omnia_eb["Sector"] == "HEAT"), "Variable_NeW"] = "PRODPHEAT"


In [ ]:
Omnia_eb.reset_index(drop=True)

### Split OMNIA regions to match with NeW regions

In [ ]:
# Load dataframe to split OMNIA regions to map with NeW regions
path_data = pathwork + "Data_Raw/Energy/"
nm_file = "Shares_tosplit_Omnia_Region.csv" # Share caulculated with raw UNDATA (see EnergyBalalnce_UNDATA file)
path = path_data + nm_file
df_reg_tosplit = pd.read_csv(path, sep=";", header = 0, dtype= str)
df_reg_tosplit.rename(columns={"Region_splitted": "Region_NeW","Sector_OMNIA": "Sector"}, inplace=True)
df_reg_tosplit["Share"] = df_reg_tosplit["Share"].astype(float) # convert into a float

In [ ]:
## Split OMNIA EB dataframe 
# List regions to split 
dict_tosplit = {"AU+WA": ["AU","WA1"], "DE+OE": ["DE", "OE1"], "FR+IT+ES+OE": ["FR","IT","ES","OE2"],
                 "ID+WA": ["ID", "WA2"], "UK+WE": ["UK","WE"]}
# Dataframe to store 
df_reg_splitted = pd.DataFrame()

# For each region to split, calculate the value for the region and the remianing, caution for "FR+IT+ES+OE", data storage in the dataframe a bit different
for agg, lst_co in dict_tosplit.items():
    df_agg = Omnia_eb.loc[Omnia_eb["Region_NeW"] == f"{agg}"].copy() # sub-set with region to split
    reg_rst = lst_co[-1] # last element of the list
    for reg in lst_co[:-1]:
        df_split = df_reg_tosplit.loc[(df_reg_tosplit["Region_NeW"] == f"{reg}")].copy() # sub-set of share dataframe
        df_merged = pd.merge(df_agg[["Variable_NeW", "Sector", "Year", "Value", "Unit"]],
                     df_split[[ "Variable_NeW", "Sector", "Share"]],
                     on = ["Variable_NeW", "Sector"], how= "left") # Merge both dataframe on all row (left) from df_agg 
        sec_mean_sh = df_merged.groupby('Sector')['Share'].transform(lambda x: x.fillna(x.mean())) # replace nan value in column "Share" by mean value by "Sector" column
        df_merged['Share'] = df_merged['Share'].fillna(sec_mean_sh) 
        df_merged.loc[df_merged['Share'].isna() & (df_merged['Value'] == 0), 'Share'] = 0 # reach remaining nan values
        df_merged.loc[df_merged['Share'].isna(), "Share"] = 0 # reach remaining nan values (case where value exists only for the second region)
        if reg_rst == "OE2": # In case region "FR+IT+ES+OE", store only "Value_rest" for France
            df_merged["Value_splitted"] = df_merged["Share"]*df_merged["Value"]
            df_merged["Value_rest"] = 0
            df_merged["Region_splitted"] = f"{reg}"
            df_merged["Region_rest"] = f"{reg_rst}"
            df_reg_splitted = pd.concat([df_reg_splitted, df_merged])
        else:
            df_merged["Value_splitted"] = df_merged["Share"]*df_merged["Value"]
            df_merged["Value_rest"] = (1-df_merged["Share"])*df_merged["Value"]
            df_merged["Region_splitted"] = f"{reg}"
            df_merged["Region_rest"] = f"{reg_rst}"
            df_reg_splitted = pd.concat([df_reg_splitted, df_merged])
            
            

# to add value_rest and value_splitted in one column
# for value_splitted
df_split_tmp = df_reg_splitted[["Variable_NeW", "Region_splitted", "Sector", "Year", "Value_splitted", "Unit"]].copy()
df_split_tmp = df_split_tmp.rename(columns={"Region_splitted": "Region_NeW", "Value_splitted": "Value"})
# for value_rest
df_rest_tmp = df_reg_splitted[["Variable_NeW", "Region_rest", "Sector", "Year", "Value_rest", "Unit"]].copy()
df_rest_tmp = df_rest_tmp.rename(columns={"Region_rest": "Region_NeW","Value_rest": "Value"})
# concatenate both
df_eb_splitted = pd.concat([df_split_tmp, df_rest_tmp], ignore_index=True)
df_eb_splitted = df_eb_splitted.groupby(by=["Variable_NeW", "Region_NeW", "Sector", "Year", "Unit"], as_index=False)["Value"].sum()



#  To calculate remaining value for region "FR+IT+ES+OE"           
df_tmp = df_reg_splitted.loc[df_reg_splitted["Region_splitted"].isin(["FR","IT","ES"])].copy() # sub-set to calculate the sum of spliited values
df_tmp = df_tmp.groupby(by=["Variable_NeW", "Sector", "Year", "Unit"], as_index=False)["Value_splitted"].sum() # sum of splitted values
df_tmp_val = df_reg_splitted.loc[df_reg_splitted["Region_splitted"].isin(["FR","IT","ES"])].copy() # sub-set
df_tmp_val = df_tmp_val[["Variable_NeW", "Sector", "Year", "Value", "Unit"]].drop_duplicates() # keep only uinqie combination of the columns

df_mrg = pd.merge(df_tmp, df_tmp_val, on=["Variable_NeW", "Sector", "Year", "Unit"], how="inner") # merge both dataframes
df_mrg["Value_rest"] = df_mrg["Value"] - df_mrg["Value_splitted"] # calculate remaining values
df_mrg["Region_NeW"] = "OE2" 
df_mrg = df_mrg[["Variable_NeW", "Region_NeW", "Sector", "Year", "Unit", "Value_rest"]]
df_mrg.rename(columns={"Value_rest": "Value"}, inplace=True)

# Suppress existing row for "OE2" & merge calculated remaining values
df_eb_splitted = df_eb_splitted.loc[df_eb_splitted["Region_NeW"] != "OE2"]
df_eb_splitted = pd.concat([df_eb_splitted,df_mrg])

# Aggregate rest by NeW regions
for reg in ["OE1", "OE2", "WA1", "WA2", "WA3"]:
    df_eb_splitted.loc[(df_eb_splitted["Region_NeW"] == f"{reg}"), "Region_NeW"] = reg[:2]
df_eb_splitted = df_eb_splitted.groupby(by=["Variable_NeW", "Region_NeW", "Sector", "Year", "Unit"], as_index=False)["Value"].sum()  

Omnia_eb_NeW = Omnia_eb.loc[~Omnia_eb["Region_NeW"].isin(["AU+WA","DE+OE","FR+IT+ES+OE","ID+WA","UK+WE"])].copy()
Omnia_eb_NeW = pd.concat([Omnia_eb_NeW, df_eb_splitted], ignore_index=True)
Omnia_eb_NeW = Omnia_eb_NeW.groupby(by=["Variable_NeW", "Region_NeW", "Sector", "Year", "Unit"], as_index=False)["Value"].sum()  

print("Percentage error between sum of all Omnia and splitted values : ", (Omnia_eb_NeW["Value"].sum()-Omnia_eb["Value"].sum())/Omnia_eb["Value"].sum()*100, "%")